# Домашнее задание

Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции. Для этого объедините 2 массива в DataFrame. Обоснуйте выбор теста на корреляцию. Сделайте вывод по гипотезе.
 
 
Также ответьте на вопрос:
Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.


In [1]:
# Исходные данные и инициализация
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]

In [2]:
import pandas as pd

### 1.Объединение двух массивов в DataFrame

In [3]:
# объединение массивов в DataFrame
df_p = pd.DataFrame({'petersburg':petersburg})
df_m = pd.DataFrame({'magadan': magadan},)

# массивы разной длины, поэтому объеденяем через JOIN

df = df_p.join(df_m,how='outer')

df

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,NaN,0.0689


Выборки разной длины, для рассчётов нужны непустые значения, воспользуемся исправленным датафреймом, где пустые значения заполнены медианным.

In [9]:
df_corr = df.fillna(df.median())
df_corr


,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,0.1016,0.0689


### 2. Проверяем нормальность

In [13]:
# проверка данных на нормальность
from scipy.stats import shapiro
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'
alpha = 0.05

print ('Тест Шапиро-Уилка:')
_, p = shapiro(df_corr)
print('p=%.3f' % p)
# Интерпретация 
if p > alpha:
	print(H0)
else:
	print(Ha)

Тест Шапиро-Уилка:
p=0.101
Данные распределены нормально


Принимаем гипотезу, что данные распределены нормально. Тест Д’Агостино не проводили, поскольку для него требуется количество данных >20

Мы имеем 2 независимые выборки (Петербург и Магадан), данные в которых являются непрерывными, нормально распределенными.


### 3. Поиск корреляции

Проверим гипотезу о том, что данные по мидиям как-то связаны между собой.<br>
Поскольку данные распределены нормально - выберем тест Пирсона. Коэффициентом Пирсона можно измерить линейную зависимость между двумя непрерывными переменными.

In [15]:
from scipy.stats import pearsonr

# Сформулируем нулевую и альтернативную гипотезы

H0 = 'Нет линейной зависимости между данными.'
Ha = 'Есть линейная зависимость между данными.'

# Определим alpha
alpha = 0.05

correlation, p_value = pearsonr(df_corr['petersburg'], df_corr['magadan'])

print("Коэффициент корреляции:", correlation)
print("p-значение:", p_value)

if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")


Коэффициент корреляции: 0.25666515110501303
p-значение: 0.5394705530383969
0.10142377018928528 > 0.05. Мы не можем отвергнуть нулевую гипотезу. Нет линейной зависимости между данными.


Проведя тест, можно утверждать, что между данными нет линейной зависимости.<br>
<i>и было-бы очень странно, если-бы она была!</i>

### 4. Сравнение выборок

Независимая переменная у нас является категориальной (Город)<br>
А зависимая переменная (размер мидии) - является количественной, и происходит из разных совокупностей<br>
При этом зависимая переменная (размер мидии) является нормально распределённой, а сами выборки имеют размер меньше 30.<br>
В таких условиях, для сравнения выборок применяется <b>Независимый t-тест</b>


In [29]:
from scipy.stats import ttest_ind

# Сформулируем нулевую и альтернативную гипотезы

H0 = 'Нет значимой разницы между размером раковины мидий.'
Ha = 'Есть значимая разница между размером раковины мидий.'

# Определим alpha
alpha = 0.05

print('\n' + "*** Результаты независимого T-теста ***")
test_results = ttest_ind(df['magadan'], df['petersburg'], equal_var=True, nan_policy='omit')
p = round(test_results[1],2)

print (p)
if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")




*** Результаты независимого T-теста ***
0.0
0.0 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между размером раковины мидий.


### 5. Вывод

Исходя из полученного теста - можно утверждать, что размеры мидий Петербурга и Магадана значимо отличаются друг от друга